<p>Twitterのプロフィールで興味・関心キーワードを抜き出したものをcorpus.pklに保存されている</p>
<p> それらのキーワードを1つ隠し、残りのキーワードから隠した1つを予測し、学習することで興味ベクトルを作る</p>

In [1]:
import pickle
with open('data/corpus.pkl', 'rb') as f:
    corpus = pickle.load(f)
with open('data/dictionary.pkl', 'rb') as f:
    dictionary = pickle.load(f)

In [2]:
reverse_dictionary = {v: k for k, v in dictionary.items()}

In [3]:
import os
def rm_logs(dir_path):
    for root, dirs, files in os.walk(dir_path, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))

In [4]:
data_index = 0
import numpy as np
from copy import copy
import random

def generate_batch(batch_size):
    global data_index
    batch = []
    labels = []
    batch_count = 0
    
    while(batch_count < batch_size):
        fun_ids = copy(corpus[data_index])
        # 興味が１つしかない場合は予測できないので無視
        if len(fun_ids) >= 2:
            label = random.sample(fun_ids, 1)
            fun_ids.remove(label[0])
            for fun_id in fun_ids:
                batch.append([batch_count, fun_id])
            labels.append(label)
            batch_count += 1
        data_index = (data_index + 1) % len(corpus)
        if batch_count >= batch_size:
            break

    return batch, labels

_batch, _labels = generate_batch(batch_size=10)
print(' batch:', [[idx, reverse_dictionary[v_id]] for idx, v_id in _batch])
print(' labels:', [[idx, reverse_dictionary[v_id[0]]] for idx, v_id in enumerate(_labels)])

 batch: [[0, '作曲家'], [0, 'スペイン語'], [0, '翻訳'], [0, 'GameBoy'], [0, '開発'], [1, 'Twit'], [1, 'しよう'], [2, 'フォロリク'], [2, 'tweet'], [2, '日常'], [2, '垢'], [3, '男'], [3, '少女'], [3, 'ハドソン'], [3, '在住'], [3, '中年'], [3, '台湾'], [3, '東方'], [3, 'コスプレ'], [3, '娘。'], [3, 'コレクター'], [3, 'レトロゲーム'], [4, '轉'], [4, '魔法'], [5, '沖縄'], [5, '作家'], [5, 'よろしくお願いします'], [5, '東野圭吾'], [5, '広東語'], [5, '小説'], [5, '日本'], [5, 'tweet'], [5, 'ファン'], [5, '推理'], [5, 'ミステリ'], [5, 'お世話'], [6, '勉強'], [7, '流通'], [7, 'ケータイ'], [7, 'マーケ'], [7, 'ネットワーク機器'], [7, '興味'], [8, 'ジャンル'], [8, '都内'], [8, '組織'], [8, '個人'], [8, 'ネタ'], [8, 'tweet'], [8, '所属'], [9, '写真'], [9, 'スルー'], [9, 'ネタ'], [9, '歓迎'], [9, 'デマ'], [9, 'ヲチ'], [9, '公式RT']]
 labels: [[0, '福岡'], [1, '興味'], [2, '鍵'], [3, '台湾人'], [4, '魔法少女'], [5, 'クイーン'], [6, '語'], [7, 'カメラ'], [8, '見解'], [9, 'お断り']]


In [5]:
# Train cbow model
import tensorflow as tf
import math
import numpy as np
import random

BATCH_SIZE = 500
EMBEDDING_SIZE = 200
NUM_SAMPLED = BATCH_SIZE//3
VOCAB_SIZE = len(dictionary)
SAMPLE_IDS = [dictionary[w] for w in ['英語', '機械学習', 'プログラミング', '将棋', 'サッカー', 'ビリヤード', '議論', '脳科学']]
SAMPLE_SIZE = len(SAMPLE_IDS)
# 辞書の単語IDは0から順に振られているのでrangeでもOK
sample_indices = np.array([[idx, vocab_id] for idx, vocab_id in enumerate(SAMPLE_IDS)])

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):
    # input data
    # sparse matrix(shape=batch_size*vocab_size)にtrain_indicesの部分に0でない興味が入り、valuesはすべて1
    train_indices = tf.placeholder(tf.int64) # 0でない場所を指す
    train_dataset = tf.SparseTensor(train_indices, values=tf.ones([tf.shape(train_indices)[0]], tf.float32), dense_shape=[BATCH_SIZE, VOCAB_SIZE])
    train_labels = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 1])
    sample_dataset = tf.SparseTensor(sample_indices, values=tf.ones([tf.shape(sample_indices)[0]], tf.float32), dense_shape=[SAMPLE_SIZE, VOCAB_SIZE])
    
    # Variables
    # 一様分布で興味ベクトル初期化
    embeddings = tf.Variable(tf.random_uniform([VOCAB_SIZE, EMBEDDING_SIZE], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal([VOCAB_SIZE, EMBEDDING_SIZE], stddev=1.0/math.sqrt(EMBEDDING_SIZE)))
    tf.summary.scalar('weights', tf.reduce_mean(softmax_weights))
    softmax_biases = tf.Variable(tf.zeros([VOCAB_SIZE]))
    # 該当ユーザーのtrainingデータの興味ベクトルをすべて足す
    embed = tf.sparse_tensor_dense_matmul(train_dataset, embeddings)

    # softmax loss with negative sampling
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(
            weights=softmax_weights,
            biases=softmax_biases,
            inputs=embed, # batch_size * embedding_size
            labels=train_labels,
            num_sampled=NUM_SAMPLED,
            num_classes=VOCAB_SIZE))
    tf.summary.scalar('loss', loss)
    optimizer = tf.train.AdagradOptimizer(0.8).minimize(loss)

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    # サンプルデータのベクトル
    sample_embed = tf.sparse_tensor_dense_matmul(sample_dataset, normalized_embeddings)
    # サンプルデータとの類似度
    similarity = tf.matmul(sample_embed, tf.transpose(normalized_embeddings))
    # tensorboard summary
    merged_summary = tf.summary.merge_all()

In [6]:
NUM_STEPS = 50000
SUMMARY_DIR = 'data/tensorboard'
TOP_K = 8

with tf.Session(graph=graph) as sess:
    rm_logs(SUMMARY_DIR)
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    tf.global_variables_initializer().run()
    average_loss = 0
    for step in range(1, NUM_STEPS+1):
        batch_data, batch_labels = generate_batch(BATCH_SIZE)
        feed_dict = {
            train_indices: batch_data,
            train_labels: batch_labels
        }
        summary, I = sess.run([merged_summary, loss], feed_dict=feed_dict)
        average_loss += I
        if step % 1000 == 0:
            average_loss = average_loss / 1000
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
            writer.add_summary(summary, step)
        if step % 5000 == 0:
            sim = similarity.eval()
            for i in range(SAMPLE_SIZE):
                sample_fun = reverse_dictionary[sample_indices[i][1]]
                nearest = (-sim[i, :]).argsort()[1:TOP_K+1]
                log = 'Nearest to %s:' % sample_fun
                for k in range(TOP_K):
                    close_fun = reverse_dictionary[nearest[k]]
                    log = '%s %s ' % (log, close_fun)
                print(log)
    writer.close()
    final_embeddings = normalized_embeddings.eval()

Average loss at step 1000: 5.989287
Average loss at step 2000: 5.952760
Average loss at step 3000: 5.946275
Average loss at step 4000: 5.879417
Average loss at step 5000: 5.965440
Nearest to 英語: アニメヲタ  ROA  ミルキアン  セシリア  エイズ  インフルエンサー  松崎梨央  天象 
Nearest to 機械学習: 道明寺歌鈴  慰安  年末年始  壱星  孖  メル友  サザン・ロック  デグー 
Nearest to プログラミング: バビロニア  主犯  応援  地道  思  あざ  末弟  ホラーコア 
Nearest to 将棋: 三代目J Soul Brothers  万歳  オデ  お座  サトー  基準  コート  周防尊 
Nearest to サッカー: グロリアス  色彩検定  キャラ変  年越し  ゆのっち  番協  北見  地球人 
Nearest to ビリヤード: ワカマツカオリ  機工  白玉粉  にっしー  エセ  えれん  笠松  旅客機 
Nearest to 議論: ソルジャー  憂鬱  ィズニーシー  Clie  フォロバアニメ  蓉  リムーブ・ブロック  白井萌花 
Nearest to 脳科学: 書道  グラスアイ  ソメワケ  百貨店  注射  京産  ゴシック  バセドウ病 
Average loss at step 6000: 5.953638
Average loss at step 7000: 5.938462
Average loss at step 8000: 5.887482
Average loss at step 9000: 5.963041
Average loss at step 10000: 5.961713
Nearest to 英語: アニメヲタ  ROA  ミルキアン  セシリア  エイズ  インフルエンサー  松崎梨央  天象 
Nearest to 機械学習: 道明寺歌鈴  慰安  年末年始  壱星  孖  メル友  サザン・ロック  デグー 
Nearest to プログラミング

KeyboardInterrupt: 

In [ ]:
print(dictionary['機械学習'])
print(dictionary['プログラミング'])
print(dictionary['将棋'])
print(dictionary['ビリヤード'])
print(dictionary['ダーツ'])
print(dictionary['旅行'])
print(dictionary['温泉'])